In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = "https://www.unigo.com/colleges/university-of-southern-california"

school_name = url.split("/")[-1]

response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, "html.parser")

In [3]:
def getOverallScore(soup):
    star_rating = soup.find('div', {'class': 'col-sm-2 star-rating'})
    percentage = star_rating.find('div', {'class': 'front-stars'})['style']
    percentage = percentage.replace('width:', '').replace('%', '').strip()
    return round(float(percentage) / 20, 2) # out of 5

print(getOverallScore(soup), "out of 5")

4.37 out of 5


In [4]:
def aboutSchool(soup):
    aboutSchoolStr = ""
    college_info_container = soup.find(
        'div', {'class': 'college-general-information-container'})
    h2_tags = college_info_container.find_all('h2')
    info_paragraphs = []

    if len(h2_tags) >= 2:
        first_h2_tag = h2_tags[0]
        second_h2_tag = h2_tags[1]

        for sibling in first_h2_tag.next_siblings:
            if sibling == second_h2_tag:
                break
            elif sibling.name == 'p':
                info_paragraphs.append(sibling.get_text())

    for p in info_paragraphs:
        aboutSchoolStr += p
    
    return aboutSchoolStr


aboutSchool(soup)


"Founded in 1880, University of Southern California. is a Private college. Located in California, which is a city setting in California, the campus itself is Urban. The campus is home to 18,794 full time undergraduate students, and 25,077 full time graduate students.The University of Southern California Academic calendar runs on a Semester basis. In the  school year the student to faculty ratio was 9:1. There are 2062 full time instructional teachers. Degrees awarded at University of Southern California include: Bachelor's Degree, Masters Degree, Post-master's certificate, Doctor's degree."

In [5]:
# helper
def get_second_number(lst):
    second_num = None
    for i in range(len(lst)):
        try:
            num = float(lst[i])
            if i != 0:
                second_num = num
                break
        except:
            pass
    return second_num

In [6]:
def seperateReviews(soup):
    reviews_anchor_tag = soup.find(
        'a', {'class': 'anchor-link', 'id': 'Reviews'})
    if reviews_anchor_tag:
        h4_tags = reviews_anchor_tag.find_all_next('h4')
        for h4 in h4_tags:
            h4_text = h4.get_text().strip()
            if h4_text.startswith('How would you rate'):
                h4_text = h4_text[len('How would you rate'):].strip()
                h4_text = h4_text[:-1].strip().capitalize()
                p_tag = h4.find_next_sibling('p')
                if p_tag:
                    p_text = p_tag.get_text().strip()
                    individualReview = get_second_number(p_text.split())
                    print(f"{h4_text}: {individualReview}")


seperateReviews(soup)


On-campus housing: 3.4
Off-campus housing: 3.0
Campus food: 3.6
Campus facilities: 4.4
Class size: 4.1
School activities: 4.4
Local services: 4.0
Academics: 4.0


In [7]:
def getIndividualOpinion(soup):
    reviews_list = soup.find("div", class_="reviews-list")

    for name in reviews_list.find_all("strong"):
        date = name.find_next_sibling("span")
        div_tag = date.find_next_sibling("div")
        rating = div_tag["data-review-count"]  # out of five
        p_tag = date.find_next_sibling("p")
        print(f"{name.text}, {date.text}, {int(rating)}, {p_tag.text}")


getIndividualOpinion(soup)


Olivia, 03/28/2023, 5, Personally, the University of Southern California has a diverse student body with plenty of resources available for students to thrive. No matter what you plan to pursue, the University of Southern California has a place for you. There interdisciplinary learning allows for students to dive deeper into other areas of study that interest them, creating well-rounded students both academically and socially.
Anna, 12/28/2022, 4, USC is a great place to network and meet new people. I am a fan of the pretty campus, the school spirit and the opportunities available for students. You are surrounded by very talented individuals, so imposter syndrome may occur. In addition, off campus housing is expensive and food near campus can be pricey.  
mathew, 04/04/2022, 4, A super fun and intense experience. It is the only metro campus I know that is small, has diversity in academic departments, and yet concentrated.
Ashley, 12/14/2021, 5, USC is very attentive to it's students. If

## FAQ Question

In [43]:
# helper
def getQuestion(soup):
    header_div = soup.find("div", class_="college-review-question-header-container")
    question = header_div.find("h2")
    return question.text


In [46]:
# helper
def getNameAndComment(soup):
    reviews = soup.find_all('div', class_='overall-college-user-review-container')
    names = []
    comments = []

    for review in reviews:
        names.append(review.find('strong').text)
        comments.append(review.find('p').text)

    return names, comments


In [48]:
def getFAQ():
    FAQ_url = [f"https://www.unigo.com/colleges/{school_name}/describe-a-typical-weekend",
               f"https://www.unigo.com/colleges/{school_name}/describe-how-your-school-looks-to-someone-whos-never-seen-it",
               f"https://www.unigo.com/colleges/{school_name}/describe-the-best-and-worst-parts-of-the-social-scene-on-campus",
               f"https://www.unigo.com/colleges/{school_name}/describe-the-dorms",
               f"https://www.unigo.com/colleges/{school_name}/describe-the-students-at-your-school",
               f"https://www.unigo.com/colleges/{school_name}/describe-your-favorite-campus-traditions",
               f"https://www.unigo.com/colleges/{school_name}/heres-your-chance-say-anything-about-your-college",
               f"https://www.unigo.com/colleges/{school_name}/is-the-stereotype-of-students-at-your-school-accurate",
               f"https://www.unigo.com/colleges/{school_name}/tell-us-about-the-food-and-dining-options",
               f"https://www.unigo.com/colleges/{school_name}/tell-us-about-the-sports-scene-on-campus",
               f"https://www.unigo.com/colleges/{school_name}/tell-us-about-your-professors",
               f"https://www.unigo.com/colleges/{school_name}/what-are-the-academics-like-at-your-school",
               f"https://www.unigo.com/colleges/{school_name}/what-are-the-most-popular-classes-offered",
               f"https://www.unigo.com/colleges/{school_name}/what-are-the-most-popular-student-activities-groups",
               f"https://www.unigo.com/colleges/{school_name}/what-are-your-classes-like",
               f"https://www.unigo.com/colleges/{school_name}/what-do-students-complain-about-most",
               f"https://www.unigo.com/colleges/{school_name}/what-is-the-stereotype-of-students-at-your-school",
               f"https://www.unigo.com/colleges/{school_name}/what-is-your-overall-opinion-of-this-school",
               f"https://www.unigo.com/colleges/{school_name}/what-should-every-freshman-at-your-school-know-before-they-start",
               f"https://www.unigo.com/colleges/{school_name}/whats-the-dating-scene-like",
               f"https://www.unigo.com/colleges/{school_name}/whats-the-greek-scene-like",
               f"https://www.unigo.com/colleges/{school_name}/whats-unique-about-your-campus",
               f"https://www.unigo.com/colleges/{school_name}/when-you-step-off-campus-what-do-you-see",
               f"https://www.unigo.com/colleges/{school_name}/where-is-the-best-place-to-get-work-done-on-campus",
               f"https://www.unigo.com/colleges/{school_name}/why-did-you-decide-to-go-to-this-school",
               f"https://www.unigo.com/colleges/{school_name}/what-is-the-stereotype-of-students-at-your-school-is-this-stereotype-accurate",
               f"https://www.unigo.com/colleges/{school_name}/whats-the-one-thing-you-wish-someone-had-told-you-about-freshman-year",
               f"https://www.unigo.com/colleges/{school_name}/whats-the-hardest-thing-about-freshman-year",
               f"https://www.unigo.com/colleges/{school_name}/i-dont-drink-will-it-be-hard-for-me-to-go-to-parties",
               f"https://www.unigo.com/colleges/{school_name}/how-often-should-i-phone-home-is-once-a-day-too-much",
               f"https://www.unigo.com/colleges/{school_name}/what-do-you-consider-the-worst-thing-about-your-school-why",
               f"https://www.unigo.com/colleges/{school_name}/what-kind-of-person-should-attend-this-school",
               f"https://www.unigo.com/colleges/{school_name}/what-kind-of-person-should-not-attend-this-school",
               f"https://www.unigo.com/colleges/{school_name}/what-do-you-brag-about-most-when-you-tell-your-friends-about-your-school",
               f"https://www.unigo.com/colleges/{school_name}/whats-the-most-frustrating-thing-about-your-school"]
    
    for url in FAQ_url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        print(getQuestion(soup))
        print(getNameAndComment(soup))
        


getFAQ()

Describe a typical weekend. 
(['Allegra', 'Bernadette'], ['Typical weekend for a non-Greek student: \r\nThursday: Happy Hour Margaritas at La Barca on Vermont with friends from work or a campus organization like the radio station or SCOutfitters. Then Tommy\'s Place, an on-campus concert venue to hear a friend\'s band play a gig. If you\'re lucky, you won\'t have any classes on Friday to worry about.\r\nFriday: Check out a film screening and Q&A in the early evening at the School of Cinematic Arts. After that, head to Menlo Ave., a street filled with students\' houses, where sketch comedy troupe Commedus Interruptus is holding a "fundrager" for their next show. \r\nSaturday: Sleep in. Grab brunch at Jacks & Joe, where they serve the most outrageous pancakes you\'ve ever had. Get some work done. Head downtown for a concert at the Wiltern, and hopefully make it back in time for the tail end of a DJ set at a Delta Kappa Alpha party (USC\'s coed cinema fraternity). \r\nSunday: Kick it into